In [16]:
##THIS I FOR ACCESSING THE CONCORDE TSP SOLVER
from subprocess import Popen
import os
class tsp:
    def __init__(self, points=None):
        self.points = points if points else []
    def solve(self, filename='_tmp_'):
        with open('%s.dat' % filename, 'w') as fp:
            fp.write('TYPE : TSP\nDIMENSION : %d\n' % len(self.points))
            fp.write('EDGE_WEIGHT_TYPE : EUC_2D\nNODE_COORD_SECTION\n')
            for i, (x, y) in enumerate(self.points):
                fp.write('%d %g %g\n' % (i, x, y))
            fp.write('EOF\n')
        prc = Popen('./concorde -m  %s.dat' % filename, shell=True)
        prc.wait()
        with open('%s.sol' % filename) as fp:
            ll = (''.join(fp)).split()
        self.result = [int(s) for s in ll[1:]]
        for s in ('%s.dat', '%s.sol'): os.remove(s % filename)

In [17]:
import geopy

import json
import pymongo
from geopy.distance import vincenty
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["bandsintown"]

# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

import numpy as np
from matplotlib import pyplot as plt


In [18]:
final_embeddings=[]

for artist in db.minedArtists.find({ 'name' : 'Deerhoof' }) :
    for tour in artist["tours"]:
        tourCoords= []
        tourdist=0
        tourdistOpt=0
        print len(tour["tourCoords"])
        if len(tour["tourCoords"])>2:
            tourCoords=[tuple(tourCoord) for tourCoord in tour["tourCoords"]]
            #print tourCoords
            t=tsp(tourCoords)
            t.solve()
            print t.result
            print t.points
            print "long",len (t.result)
            i = 0
            while i < len (t.result)-1:
                res=t.result[i]
                res2=t.result[i+1]
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdistOpt= tourdistOpt + vincenty(t.points[res],t.points[res2]).kilometers
                i+=1
            j = 0
            while j < len (tourCoords)-1:
                
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdist= tourdist + vincenty(tourCoords[j],tourCoords[j+1]).kilometers
                j+=1
            
            print "DEFAULT TOUR DIST",tourdist
            print "TOURDISTOPTIMIZED",tourdistOpt
            


8
[0, 3, 4, 6, 5, 7, 2, 1]
[(29.7630556, -95.3630556), (32.78465, -96.808121), (36.1538889, -95.9925), (36.15251, -86.784858), (39.083035, -84.509825), (43.6350968, -79.4157243), (42.982846, -81.252275), (42.636241, -83.291617)]
long 8
DEFAULT TOUR DIST 2951.72940395
TOURDISTOPTIMIZED 4152.43106033
3
[0, 1, 2]
[(44.9677429, -93.289093), (39.74015, -104.948372), (40.7563, -111.899109)]
long 3
DEFAULT TOUR DIST 1722.72567637
TOURDISTOPTIMIZED 1722.72567637
2
2
22
[0, 4, 5, 6, 9, 20, 21, 19, 18, 14, 17, 16, 13, 15, 12, 11, 10, 8, 7, 3, 2, 1]
[(34.102718, -118.327156), (37.794379, -122.420596), (45.540728, -122.663737), (47.614065, -122.320401), (40.763776, -111.876661), (39.74015, -104.948372), (41.266524, -95.934431), (44.978633, -93.276107), (43.020287, -87.915123), (39.166572, -86.533332), (41.9498429, -87.6586213), (42.636591, -83.291778), (40.708471, -74.00295), (40.7141667, -74.0063889), (35.881268, -78.597115), (42.318427, -72.627827), (39.960044, -75.153442), (38.917913, -77.02362

[0, 1, 2]
[(36.142343, -115.156716), (34.077803, -118.260064), (32.224261, -110.974272)]
long 3
DEFAULT TOUR DIST 1073.90218996
TOURDISTOPTIMIZED 1073.90218996
2
10
[0, 2, 5, 4, 6, 7, 8, 9, 3, 1]
[(40.6970556, -73.9345632), (40.6970556, -73.9345632), (42.352865, -71.132547), (39.961702, -75.155499), (45.519013, -122.652089), (47.614065, -122.320401), (37.794379, -122.420596), (34.0518408, -118.2670763), (32.730196, -117.171902), (32.222229, -110.966895)]
long 10
DEFAULT TOUR DIST 7255.92595769
TOURDISTOPTIMIZED 10163.7956591
2
12
[0, 11, 10, 9, 8, 7, 6, 5, 4, 2, 3, 1]
[(39.716388, -104.987688), (41.2586111, -95.9375), (41.925898, -87.6498), (43.067286, -87.904847), (39.7526703, -86.1409912), (38.04055, -84.478761), (36.152824, -86.780319), (33.958264, -83.377113), (30.412706, -87.215186), (29.9348836, -90.1075921), (29.77006, -95.357559), (32.784768, -96.784476)]
long 12
DEFAULT TOUR DIST 4617.66813431
TOURDISTOPTIMIZED 4771.17453277
11
[0, 7, 8, 6, 10, 9, 3, 5, 4, 1, 2]
[(52.501585, 1